In [8]:
!pip uninstall -y numpy pandas
!pip install pandas==2.2.2 numpy==1.26.4 alpaca-py ta xgboost matplotlib tqdm python-dotenv yfinance scikit-learn joblib --quiet --disable-pip-version-check --no-deps


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2


In [9]:
import os
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
from tqdm import tqdm

# Yahoo Finance data
import yfinance as yf

# Technical indicators
import ta

# Model
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Visualization
import matplotlib.pyplot as plt

# For environment variables (optional if storing ticker or API keys)
from dotenv import load_dotenv
load_dotenv()


False

In [15]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

symbol = "AAPL"
interval = "1m"
days_back = 60  # target history

def fetch_1min_yahoo(symbol, days_back):
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days_back)
    data_all = []

    # Yahoo allows only ~7 days of 1-min data at a time
    chunk = 6  # fetch 6 days per request to be safe
    current_start = start_date

    while current_start < end_date:
        current_end = min(current_start + timedelta(days=chunk), end_date)
        df_chunk = yf.download(
            symbol,
            interval=interval,
            start=current_start.strftime("%Y-%m-%d"),
            end=current_end.strftime("%Y-%m-%d"),
            progress=False
        )
        if not df_chunk.empty:
            data_all.append(df_chunk)
        current_start = current_end

    if data_all:
        df = pd.concat(data_all)
        df = df[~df.index.duplicated(keep="first")]
        df.sort_index(inplace=True)
        return df
    else:
        return pd.DataFrame()

df_1min = fetch_1min_yahoo(symbol, days_back=25)
print("Fetched:", df_1min.shape)
df_1min.head()


/tmp/ipython-input-2119337512.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_chunk = yf.download(
/tmp/ipython-input-2119337512.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_chunk = yf.download(
/tmp/ipython-input-2119337512.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_chunk = yf.download(
/tmp/ipython-input-2119337512.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_chunk = yf.download(
/tmp/ipython-input-2119337512.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_chunk = yf.download(


Fetched: (7385, 5)


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Datetime,,,,,
2025-07-21 13:30:00+00:00,211.899994,212.250000,211.640900,212.050003,3199258
2025-07-21 13:31:00+00:00,212.250595,212.300003,211.759995,211.940002,404437
2025-07-21 13:32:00+00:00,212.554993,212.639999,212.169998,212.240005,405166
2025-07-21 13:33:00+00:00,212.345001,212.689407,212.199997,212.544998,260261
2025-07-21 13:34:00+00:00,212.909302,212.940002,212.110001,212.330002,432155
